In [1]:
import sys
#!conda install --yes --prefix {sys.prefix} pandas

In [2]:
import cv2
import glob
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt

In [3]:
def img_resize(img, scale):
    height, width, depth = img.shape
    newY, newX = int(height/scale), int(width/scale)
    img_resized = cv2.resize(img,(newX, newY))
    return img_resized 

In [4]:
def color_avg(img):    
    flat = img.reshape(-1, img.shape[-1])
    nonzero_pixel = flat[np.any(flat,axis=1)]
    return np.mean(nonzero_pixel,axis=0).tolist()

In [5]:
def contour_by_color(img,boundaries,kernal_size,contour_number=1):
    mask = mask_contour = np.zeros(img.shape[:2], dtype="uint8")
    for (lower, upper) in boundaries:
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")

        # find the colors within the specified boundaries and apply
        # the mask
        mask += cv2.inRange(img, lower, upper) 
        kernel = np.ones((kernal_size,kernal_size), np.uint8)  # note this is a horizontal kernel
        d_im = cv2.dilate(mask, kernel, iterations=1)
        mask_closed = cv2.erode(d_im, kernel, iterations=1) 
        #output = cv2.bitwise_and(hsv, hsv, mask = mask)
    
    # note setting cv2.RETR_EXTERNAL to retrieve outside contour only
    _,cnts,_ = cv2.findContours(mask_closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contour_number != 1:
        contour_number = len(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:contour_number]
   
    cnt_area_all = []
    front_contour = np.zeros(img.shape[:2], dtype="uint8")
    back_contour = np.ones(img.shape[:2], dtype="uint8")*255
    for cnt in cnts:
        cnt_area = cv2.contourArea(cnt)
        if cnt_area > 150:
            cnt_area_all.append(cnt_area)
            cv2.drawContours(front_contour, [cnt], -1, (255,255,255), -1)            
            cv2.drawContours(back_contour, [cnt], -1, 0, 5)

    return front_contour, back_contour, cnt_area_all
    # masking
    ##imask = mask>0
    #ylw = np.zeros_like(hsv, np.uint8)
    #ylw[imask] = img[imask]
    
    #show image with matplotlib
    #plt.imshow(imask[:,:,[2,1,0]])
    #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    #plt.show()


In [199]:
path=os.getcwd()
img_folder = os.path.join(path,"result_img") + "/"
img_file=glob.glob('*.tiff')
img_result =  pd.DataFrame(columns=['file_name','spot_ng_ratio','ng_area','spot_area_all','num_spot','spot_area_mean','spot_area_max','ng_R','ng_G','ng_B','spot_R','spot_G','spot_B'])
#img_file = ['LF10_10.tiff']
for i in range(len(img_file)):
    print(img_file[i])
    img = cv2.imread(img_file[i])
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    
    ### find nectar guide area
    ng_boundaries = [((10,140,180), (30, 255, 255))]

    ng_front,ng_back,ng_area = contour_by_color(hsv,ng_boundaries,kernal_size=100)
    bgr_ng_front = cv2.bitwise_and(img, img, mask = ng_front)
    hsv_ng_front = cv2.bitwise_and(hsv, hsv, mask = ng_front)
    bgr_ng_back = cv2.bitwise_and(img, img, mask = ng_back)
    ng_B,ng_G,ng_R = color_avg(bgr_ng_front)
    #cv2.circle(bgr_ng_front,(int(bgr_ng_front.shape[1]/2),int(bgr_ng_front.shape[0]/2)),10,color=0,thickness=-1)
    #imshow_resize(bgr_ng_front, 10)
    
    ### find spots in nectar guide area
    spot_boundaries = [((1,110,130), (19, 230, 255))]

    spot_front, spot_back, spot_area = contour_by_color(hsv_ng_front,spot_boundaries,kernal_size=5,contour_number=2)
    bgr_spot_front = cv2.bitwise_and(bgr_ng_front, bgr_ng_front, mask = spot_front)
    bgr_spot_back = cv2.bitwise_and(bgr_ng_back, bgr_ng_back, mask = spot_back)
    spot_B,spot_G,spot_R = color_avg(bgr_spot_front)

    img_result.loc[i] = [img_file[i],sum(spot_area)/ng_area[0],ng_area[0],sum(spot_area),len(spot_area),np.mean(spot_area),max(spot_area),ng_R,ng_G,ng_B,spot_R,spot_G,spot_B]
    
    bgr_ng_front_resized = img_resize(bgr_ng_front,10)
    bgr_spot_front_resized = img_resize(bgr_spot_front,10)
    bgr_spot_back_resized = img_resize(bgr_spot_back,6)
    #cv2.imshow("bgr_ng_front_resized",bgr_ng_front_resized)
    #cv2.imshow("bgr_spot_front_resized",bgr_spot_front_resized)
    #cv2.imshow("bgr_spot_back_resized",bgr_spot_back_resized)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #cv2.waitKey(1)
    
    filename_base = img_file[i].split('.')[0]
    cv2.imwrite(img_folder + filename_base+"_NG.png",bgr_ng_front_resized)
    cv2.imwrite(img_folder + filename_base+"_SPOT.png",bgr_spot_front_resized)
    cv2.imwrite(img_folder + filename_base+"_BOTH.png",bgr_spot_back_resized)

img_result.to_csv("spot_counting.csv", encoding='utf-8', index=False)   

rto_1.tiff
RTORNAi8_1.tiff
LF10_7.tiff
LF10_6.tiff
RTO RNAi4_1.tiff
RTORNAi6_1.tiff
F1_1.tiff
RTO RNAi20_1.tiff
F1_3.tiff
RTO RNAi4_4.tiff
RTO RNAi20_3.tiff
RTORNAi8_5.tiff
rto_5.tiff
RTORNAi6_4.tiff
LF10_10.tiff
rto_4.tiff
RTORNAi6_5.tiff
RTORNAi8_4.tiff
RTO RNAi20_2.tiff
RTO RNAi4_5.tiff
F1_2.tiff
RTO RNAi20_5.tiff
LF10_9.tiff
RTORNAi8_3.tiff
RTORNAi6_2.tiff
rto_3.tiff
F1_5.tiff
RTO RNAi4_2.tiff
RTO RNAi4_3.tiff
F1_4.tiff
RTORNAi6_3.tiff
rto_2.tiff
RTORNAi8_2.tiff
LF10_8.tiff
RTO RNAi20_4.tiff


In [74]:
# define the list of boundaries
#ng_boundaries = [((10,140,180), (20, 200, 255)),
#                 ((20,200,180), (30, 255, 255))]

ng_boundaries = [((10,140,180), (30, 255, 255))]

ng_front,ng_back,ng_area = contour_by_color(hsv,ng_boundaries,kernal_size=100)
bgr_ng_front = cv2.bitwise_and(img, img, mask = ng_front)
hsv_ng_front = cv2.bitwise_and(hsv, hsv, mask = ng_front)
bgr_ng_back = cv2.bitwise_and(img, img, mask = ng_back)
hsv_ng_back = cv2.bitwise_and(hsv, hsv, mask = ng_back)
#cv2.circle(bgr_ng_front,(int(bgr_ng_front.shape[1]/2),int(bgr_ng_front.shape[0]/2)),5,color=0,thickness=-1)

imshow_resize(bgr_ng_front, 10)
#cv2.imwrite("LF10_10_NG.tiff",bgr_masked)


[5570497.5]


In [75]:
# define spots color boundaries
spot_boundaries = [((1,110,130), (19, 230, 255))]

spot_front, spot_back, spot_area = contour_by_color(hsv_ng_front,spot_boundaries,kernal_size=5,contour_number=2)
bgr_spot_front = cv2.bitwise_and(bgr_ng_front, bgr_ng_front, mask = spot_front)
hsv_spot_front = cv2.bitwise_and(hsv_ng_front, hsv_ng_front, mask = spot_front)

bgr_spot_back = cv2.bitwise_and(bgr_ng_back, bgr_ng_back, mask = spot_back)
hsv_spot_back = cv2.bitwise_and(hsv_ng_back, hsv_ng_back, mask = spot_back)

imshow_resize(bgr_spot_back, 6)
#cv2.imwrite("LF10_10_NG_spot.tiff",bgr_spot_front)

count number 1577
[87094.5, 20956.5, 18059.5, 11456.5, 10640.5, 9408.0, 8152.0, 6859.5, 6075.0, 5772.0, 5722.0, 5468.0, 5454.5, 5055.5, 4468.0, 4344.0, 3872.0, 3499.5, 3428.0, 3355.0, 3342.0, 3115.0, 2953.5, 2850.0, 2755.0, 2728.5, 2681.0, 2626.0, 2508.5, 2412.0, 2312.0, 2266.5, 2238.0, 2200.0, 2157.5, 2024.0, 2004.0, 1991.5, 1953.0, 1791.0, 1754.0, 1746.5, 1744.0, 1719.5, 1563.5, 1539.5, 1477.5, 1458.0, 1458.0, 1442.0, 1423.0, 1410.0, 1368.5, 1339.0, 1335.0, 1313.5, 1295.5, 1277.0, 1261.5, 1223.0, 1222.0, 1162.5, 1159.5, 1141.5, 1104.0, 1097.5, 1076.0, 1049.5, 1038.0, 1007.0, 976.5, 968.0, 956.0, 942.5, 923.0, 899.5, 887.0, 879.5, 869.5, 867.0, 866.5, 856.0, 823.0, 815.0, 809.5, 796.5, 790.5, 776.5, 768.0, 763.5, 758.5, 750.5, 750.0, 747.0, 747.0, 743.5, 732.5, 722.5, 706.0, 700.5, 688.0, 683.0, 680.5, 677.5, 677.5, 676.0, 663.5, 663.0, 660.0, 633.5, 617.5, 617.0, 610.0, 605.0, 601.0, 600.0, 599.0, 594.0, 592.5, 591.0, 586.5, 566.5, 564.5, 564.5, 564.5, 562.0, 558.5, 556.5, 549.0, 544

In [159]:
img_result[i]

0     LF10_10.tiff
1      [5570497.5]
2              324
3           402764
4           1243.1
5          87094.5
6          62.6434
7          187.557
8           226.48
9          67.9675
10         150.797
11         205.901
Name: 0, dtype: object

In [172]:
filename_base = img_file[i].split('.')[0]
cv2.imwrite(img_folder + "/" + filename_base+"_NG.png",bgr_ng_front_resized)

True

In [173]:
img_folder = os.path.join(path,"result_img") + "/"

In [181]:
img_file=glob.glob('*.tiff')

In [182]:
img_file

[]

In [185]:
path=os.getcwd()
img_folder = os.path.join(path,"result_img") + "/"
img_file=glob.glob('*rto*.tiff')
img_file

['rto_1.tiff', 'rto_5.tiff', 'rto_4.tiff', 'rto_3.tiff', 'rto_2.tiff']